In [38]:
%matplotlib inline

import numpy as np
import random, csv
from math import sqrt

def read_file(filename):
    with open(filename, 'r') as file:
        lines = list(csv.reader(file))
    return lines

fires = read_file('forestfires.csv')

Сделаем следующие **преобразования данных**:
* уберем из рассмотрения поле day 
* добавим новую координату для свободного члена
* переведем нечисловые координаты в числовые
* перемешаем данные

In [39]:
fires = fires[1:]

for fire in fires:
    fire.pop(3) # remove day
    fire.append(1.0) # add free term

for i in range(len(fires)):
    fires[i][2] = 1.0 if fires[i][2] in ['jun', 'jul', 'aug'] else 0.0
    fires[i] = [float(number) for number in fires[i]]

random.shuffle(fires)

Разобьем данные в соотношении 7:3. **Построим регрессионную модель**, применим модель ко второй части выборки и посчитаем
среднеквадратичную ошибку.

Далее применим к координате area преобразование  $f(x) = ln(c+x)$ для различных $c$. Посчитаем среднеквадратичную ошибку для преобразованных и для исходных значений.

In [40]:
def standart_deviation(function, sd_function):
    learn = fires[:(len(fires) * 7 // 10)]
    z_learn = np.matrix([fire[:-2] + fire[-1:] for fire in learn]) # [-2] - area
    x_learn = np.matrix([function(fire[-2]) for fire in learn]).transpose()
    theta_ls = np.linalg.inv(
        z_learn.transpose() * z_learn) * z_learn.transpose() * x_learn
    
    test = fires[(len(fires) * 7 // 10):]
    z_test = np.matrix([fire[:-2] + fire[-1:] for fire in test])
    x_test = np.matrix([function(fire[-2]) for fire in test]).transpose()
    result = z_test * theta_ls
    
    return sqrt(np.mean(np.array(
            (sd_function(result) - sd_function(x_test)).transpose()[0]) ** 2))

print("x:", "%.4f" % standart_deviation(lambda x: x, lambda x: x), "\n")
for c in range(1, 100, 10):
    print("Преобразованных ln(" + str(c) + " + x):", "%.4f" % standart_deviation(
            lambda x: np.log(c + x), lambda x: np.log(c + x)))
    print("Исходных к ln(" + str(c) + " + x):", "%.4f" % standart_deviation(
            lambda x: np.log(c + x), lambda x: np.exp(x) - c))
    print("\n")

x: 37.2429 

Преобразованных ln(1 + x): 0.6447
Исходных к ln(1 + x): 34.6908


Преобразованных ln(11 + x): 0.0430
Исходных к ln(11 + x): 34.2242


Преобразованных ln(21 + x): 0.0186
Исходных к ln(21 + x): 34.1487


Преобразованных ln(31 + x): 0.0109
Исходных к ln(31 + x): 34.1387


Преобразованных ln(41 + x): 0.0073
Исходных к ln(41 + x): 34.1539


Преобразованных ln(51 + x): 0.0053
Исходных к ln(51 + x): 34.1809


Преобразованных ln(61 + x): 0.0040
Исходных к ln(61 + x): 34.2141


Преобразованных ln(71 + x): 0.0032
Исходных к ln(71 + x): 34.2504


Преобразованных ln(81 + x): 0.0026
Исходных к ln(81 + x): 34.2882


Преобразованных ln(91 + x): 0.0022
Исходных к ln(91 + x): 34.3266




Видим, что и в случае без преобразования, и в случае с преобразованием среднеквадратичная ошибка велика, но при малых
$c$ преобразование дает чуть лучшие предсказания.

Найдем $c$, при котором **среднеквадратичное отклонение минимально**.

In [41]:
sd_array = [standart_deviation(lambda x: np.log(c + x), 
                               lambda x: np.exp(x) - c)
            for c in np.arange(0.1, 100, 0.1)]
cmin = np.argmin(sd_array) / 10 - 1
print("c = " + str(cmin))
print("%.2f" % sd_array[np.argmin(sd_array)])

c = 27.6
34.14


Теперь попробуем разбить выборку на части 7:3 **разными способами** при полученном выше $c$.

In [42]:
for _ in range(100):
    random.shuffle(fires)
    print("%.2f" % standart_deviation(lambda x: np.log(cmin + x), 
                       lambda x: np.exp(x) - cmin))

89.18
24.50
16.35
66.10
22.46
87.47
67.88
21.92
34.10
89.24
33.04
25.41
68.82
28.57
65.88
91.69
61.89
93.29
105.67
23.38
27.24
38.21
110.65
105.76
66.56
91.33
91.24
88.39
22.43
25.01
30.38
27.14
29.92
24.37
90.93
62.95
13.39
20.32
17.41
31.75
91.58
109.14
89.23
20.87
19.23
18.35
65.93
27.05
32.79
64.19
17.26
30.22
31.91
21.55
66.57
108.12
107.23
31.23
91.49
28.09
106.98
66.12
88.89
91.75
88.50
19.59
29.11
89.63
12.90
29.82
88.54
65.46
92.96
27.34
62.27
108.59
90.45
20.99
20.59
64.12
30.67
63.42
88.18
25.47
89.10
89.05
89.36
26.10
29.01
15.10
32.32
28.54
66.91
88.83
88.66
30.00
26.68
88.54
33.69
89.83


Видим, что в любом случае среднеквадратичное отклонение велико. К тому же, среднеквадратичное отклонение очень сильно зависит от способа разбиения. Поэтому линейная регрессия работает плохо.